In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 18 21:17:40 2023

@author: HERO
"""



import cv2
import numpy as np

# Kalman Filter 변수 초기화
dt = 1/30.0
kf = cv2.KalmanFilter(4, 2, 0)
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kf.transitionMatrix = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kf.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 10, 0], [0, 0, 0, 10]], np.float32) * 1e-5

# Optical Flow 변수 초기화
lk_params = dict(winSize=(15, 15), maxLevel=4, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# 비디오 캡처 초기화
cap = cv2.VideoCapture("C:/Users/HERO/Downloads/videoplayback.mp4")

# 초기 프레임 읽기
_, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(gray, 100, 0.01, 5, blockSize=3, useHarrisDetector=True, k=0.03)
prev_gray = gray.copy()

# 추적 결과 저장을 위한 빈 리스트 초기화
trajectory = []

while True:
    # 다음 프레임 읽기
    _, frame = cap.read()
    if frame is None:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Optical Flow를 이용하여 물체 위치 추적
    curr_pts, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **lk_params)
    if curr_pts is None:
        break

    # 추적 결과 중 유효한 것만 선택
    valid_curr_pts = curr_pts[st == 1]
    valid_prev_pts = prev_pts[st == 1]

    if len(valid_curr_pts) == 0:
        break

    # Kalman Filter를 이용하여 추적한 물체 위치에 대한 예측값 계산
    x, y, vx, vy = kf.predict()
    prediction = np.array([x, y], np.float32)

    # Kalman Filter를 이용하여 추적한 물체 위치에 대한 실제값 계산
    measurement = np.mean(valid_curr_pts, axis=0)
    if not np.isnan(measurement[0]) and not np.isnan(measurement[1]):
        kf.correct(measurement)

    # 예측값을 저장
    trajectory.append(prediction)

    # 추적 결과 시각화
    for i, (curr_pt, prev_pt) in enumerate(zip(valid_curr_pts, valid_prev_pts)):
        x1, y1 = curr_pt.ravel()
        x2, y2 = prev_pt.ravel()
        cv2.line(frame, (int(x1),int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.circle(frame, (int(x1), int(y1)), 5, (0, 255, 0), -1)
    # 예측 경로 시각화
    for i in range(len(trajectory)-1):
        curr_pt = (trajectory[i])
        next_pt = np.array(trajectory[i+1])
        cv2.line(frame, int(curr_pt), next_pt, (255, 0, 0), 2)
    
    # 현재 프레임의 정보를 이전 프레임 정보로 업데이트
    prev_gray = gray.copy()
    prev_pts = valid_curr_pts.reshape(-1, 1, 2)
    
    # 영상 출력
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()